# Metric Analysis for ECON model

In [19]:
# from common.metrics.gradient import Gradient
import sys
import torch
import torchinfo

sys.path.append('../../../../workspace/')

from models.econ.code.q_autoencoder import AutoEncoder

ModuleNotFoundError: No module named 'telescope_pt'

Select the desired ECON model:

In [ ]:
base_path = "/loss_landscape/checkpoint/different_knobs_subset_10/"
batch_size = 16
learning_rate = 0.1
precision = 2
size = 'small'

In [ ]:
def load_model(path, batch_size, learning_rate, precision, size):
    model_path = base_path + f'bs{batch_size}_lr{learning_rate}/ECON_{precision}b/{size}/net_1_best.pkl'
    model = AutoEncoder(
        quantize=(precision < 32),
        precision=[
            precision,
            precision,
            precision+3
        ],
        learning_rate=learning_rate,
        econ_type=size
    )
    model_param = torch.load(model_path)
    model.load_state_dict(model_param['state_dict'])
    return model


model = load_model(base_path, batch_size, learning_rate, precision, size)
torchinfo.summary(model, input_size=(1, 1, 8, 8))  # (B, C, H, W)            

/loss_landscape/checkpoint/different_knobs_subset_10/bs16_lr0.1/ECON_2b/small/net_1_best.pkl
